In [173]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [174]:
graph, random_solution, total_edge_length = dg.read_instance('data\data15.txt')
num_nodes = len(graph)

In [175]:
def tabu_swap_neighbors(solution):
    neighbors = []

    for i in range(len(solution) - 1):
        neighbor = solution.copy()
        neighbor[i], neighbor[i + 1] = neighbor[i + 1], neighbor[i]
        neighbors.append(neighbor)
        
    return neighbors

In [176]:
def tabu_swap(solution):
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            solution = solution.copy()
            tmp = solution[i]
            solution[i] = solution[j]
            solution[j] = tmp
            solutions.append(solution)
            
    return solutions

In [177]:
def tabu_inverse_complete(solution):
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            new_solution = solution.copy()
            new_solution[i:j] = reversed(new_solution[i:j])
            solutions.append(new_solution)
            
    return solutions

In [178]:
def tabu_inverse(solution):
    solutions = []

    for i in range(len(solution) - 1):
        new_solution = solution.copy()
        j = random.randrange(i+1, len(solution))
        new_solution[i:j] = reversed(new_solution[i:j])
        solutions.append(new_solution)
        
    return solutions

In [179]:
def tabu_scramble(solution):
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            new_solution = solution.copy()
            sub = new_solution[i:j]
            random.shuffle(sub)
            new_solution[i:j] = sub
            solutions.append(new_solution)
            
    return solutions

In [180]:
def tabu_scramble_continuous(solution):
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            solution = solution.copy()
            sub = solution[i:j]
            random.shuffle(sub)
            solution[i:j] = sub
            solutions.append(solution)
            
    return solutions

In [181]:
def tabu_double_bridge_move(solution, num_neighbors=1):
    n = len(solution)
    if n < 4:
        return [solution]

    neighbors = []
    for _ in range(num_neighbors):
        i = random.randint(1, n - 3)
        j = random.randint(i + 1, n - 2)
        k = random.randint(j + 1, n - 1)

        # Ensure i, j, and k are distinct
        while i == j or j == k or i == k:
            i = random.randint(1, n - 3)
            j = random.randint(i + 1, n - 2)
            k = random.randint(j + 1, n - 1)

        neighbor = (
            solution[:i] + solution[j:k] + solution[i:j] + solution[k:]
        )
        neighbors.append(neighbor)

    return neighbors

In [182]:
#TODO rename neighbors into solutions so it makes more sense
def tabu_search(graph, solution, max_iterations, neighbor_gen_func, tabu_tenure):
    num_nodes = len(graph)
    current_solution = deepcopy(solution)
    current_value = dg.calculate_total_edge_length(graph, current_solution)
    best_solution = current_solution.copy()
    best_value = current_value
    tabu_list = []

    best_i = -1

    for ind in range(max_iterations):
        neighbors = neighbor_gen_func(current_solution)
        neighbors.sort(key=lambda solution: dg.calculate_total_edge_length(graph, solution))
        #print(neighbors[0], calculate_total_edge_length(graph, neighbors[0]))

        # Find the best non-tabu neighbor
        next_solution = None
        for neighbor in neighbors:
            if neighbor not in tabu_list:
                next_solution = neighbor
                break

        # If all neighbors are tabu, choose the best one
        if next_solution is None:
            next_solution = neighbors[0]

        # Update tabu list
        tabu_list.append(next_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

        # Update current solution
        current_solution = deepcopy(next_solution)
        current_value = dg.calculate_total_edge_length(graph, current_solution)

        # Update best solution
        if current_value < best_value:
            #print(current_value)
            best_i = ind
            #print('promena best_i', best_i)
            best_solution = deepcopy(current_solution)
            best_value = current_value

    return best_solution, best_value, best_i

In [183]:
sols = []
values = []
iters = []
times = []

In [184]:
start = time.time()
solution, value, iter = tabu_search(graph, random_solution, 500, tabu_swap_neighbors, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[10, 5, 12, 2, 7, 4, 13, 14, 1, 8, 0, 6, 9, 3, 11] 186 32


In [185]:
start = time.time()
solution, value, iter = tabu_search(graph, random_solution, 200, tabu_swap, 10)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[10, 5, 12, 14, 7, 2, 1, 13, 8, 4, 0, 9, 6, 3, 11] 198 5


In [186]:
solution, value, iter = tabu_search(graph, random_solution, 200, tabu_inverse_complete, 20)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[10, 5, 12, 2, 7, 4, 13, 14, 1, 8, 11, 6, 3, 9, 0] 189 18


In [187]:
solution, value, iter = tabu_search(graph, random_solution, 500, tabu_inverse, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[10, 5, 12, 2, 7, 4, 13, 1, 14, 8, 11, 6, 3, 9, 0] 189 75


In [188]:
solution, value, iter = tabu_search(graph, random_solution, 200, tabu_scramble, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[10, 5, 12, 2, 7, 4, 13, 14, 1, 8, 11, 6, 3, 9, 0] 189 8


In [189]:
solution, value, iter = tabu_search(graph, random_solution, 300, tabu_scramble_continuous, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[11, 14, 1, 8, 6, 13, 9, 3, 12, 4, 5, 2, 7, 10, 0] 201 92


In [190]:
solution, value, iter = tabu_search(graph, random_solution, 500, tabu_double_bridge_move, 15)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[7, 5, 10, 1, 13, 4, 12, 6, 8, 2, 14, 9, 11, 3, 0] 226 144


In [196]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [tabu_swap_neighbors, tabu_swap, tabu_inverse_complete, tabu_inverse, tabu_scramble, tabu_scramble_continuous, tabu_double_bridge_move]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1))  
df.to_csv('comparison_tables/tabu_search.csv', mode='a', header=not pd.io.common.file_exists('data.csv'), index=False)

,Dim,Method,Value,Time
0,15,tabu_swap_neighbors,186,0.41
1,15,tabu_swap,198,0.99
2,15,tabu_inverse_complete,189,2.07
3,15,tabu_inverse,189,2.48
4,15,tabu_scramble,189,3.69
5,15,tabu_scramble_continuous,201,5.35
6,15,tabu_double_bridge_move,226,5.44


In [192]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: tabu_swap_neighbors 186 0.41 32


,Dim,Method,Value,Time,Best_Iter
0,15,tabu_swap_neighbors,186,0.41,32


In [193]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_best = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_best)

average: [196.86, 2.92, 53.43]


,Dim,Value,Time,Best_Iter
0,15,196.86,2.92,53.43
